### 텐서와 numpy 배열의 차이
텐서는 GPU나 다른 하드웨어 가속기에서 실행할 수 있으며 자동 미분에 최적화되어 있다.

### 신경망 모델 구성하기


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

c:\Users\User\anaconda3\envs\kaggle\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available
    else "cpu"
)

print(device)

cuda


In [3]:
# 신경망 모델을 nn.Module의 하위 클래스로 정의하고 __init__에서 신경망 계층 초기화하기

class NueralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits
    

In [4]:
# neuralnet의 isntance 생성하고 이를 device로 이동
model = NueralNetwork().to(device)
print(model)

NueralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [5]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([6], device='cuda:0')


In [6]:
# 28*28 크기의 이미지 3개로 구성된 미니배치
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [7]:
# flatten. 이때 미니배치 차원은 유지됨
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [8]:
# nn.Linear: 저장된 weight, bias를 사용해 input에 linear transform 적용
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [9]:
# activation func을 통해 nonlinearilty 도입
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.0911, -0.4736, -0.0653, -0.4180,  0.0348, -0.3630, -0.0156, -0.8920,
         -0.1365,  0.1691, -0.0232, -0.1918,  0.2253, -0.1782,  0.3599, -0.2883,
          0.0873,  0.5708,  0.4204,  0.6256],
        [-0.2539, -0.4483, -0.0250, -0.0581,  0.3076, -0.3002, -0.1238, -0.4495,
         -0.2826,  0.0555,  0.3310,  0.2120, -0.0643,  0.0129,  0.4192, -0.3004,
         -0.1513,  0.1325,  0.3185,  0.3299],
        [-0.1007, -0.2390, -0.0670, -0.5566,  0.2085,  0.0905,  0.1149, -0.6573,
          0.0411,  0.1343, -0.1637, -0.3354,  0.0332, -0.1246,  0.2771, -0.5828,
         -0.5087,  0.6395,  0.2653,  0.4743]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0348, 0.0000, 0.0000, 0.0000, 0.0000,
         0.1691, 0.0000, 0.0000, 0.2253, 0.0000, 0.3599, 0.0000, 0.0873, 0.5708,
         0.4204, 0.6256],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.3076, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0555, 0.3310, 0.2120, 0.0000, 0.0129, 0.41

In [10]:
# nn.Sequential: 순서를 갖는 모듈 컨테이너
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [11]:
# softmax
softmax = nn.Softmax(dim=1) # dim=1은 합이 1인 것을 의미
pred_probab = softmax(logits)

In [12]:
# 각 parameter에 접근해서 출력
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NueralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0020,  0.0341,  0.0081,  ..., -0.0118,  0.0140, -0.0308],
        [ 0.0126, -0.0182,  0.0182,  ...,  0.0234, -0.0236,  0.0021]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([-0.0199,  0.0035], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0262, -0.0089, -0.0264,  ...,  0.0178, -0.0121,  0.0233],
        [-0.0164,  0.0289, -0.0102,  ..., -0.0184,  0.0170, -0.0387]],
       device='cuda:0', grad_fn=<Sl